In [3]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[1],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1048)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D
from tensorflow.keras.layers import Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import gc

# Raw Data Loading
df = pd.read_csv('./data/mnist/train.csv')


# Data Split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df.drop('label', 
                         axis=1, 
                         inplace=False), 
                 df['label'], 
                 test_size=0.3, 
                 random_state=0)
# test_size : test set의 비율 (0.3 => 30%)
# random_state : split할 때 랜덤하게 split하게 되는데 이를 일정하게 고정(seed의 개념)

# Min-Max Normalization
scaler = MinMaxScaler()   # scaler = StandardScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train
del x_data_test

##### Tensorflow 2.x implementation #####
with tf.device('/device:GPU:1'):
    
    model = Sequential()

    # Conv2D(필터개수, kernel_size, activation='relu')
    model.add(Conv2D(filters=32,
                     kernel_size=(3,3),
                     activation='relu', 
                     input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=64,
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=64,
                     kernel_size=(3,3),
                     activation='relu'))

    model.add(Flatten())
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=256, 
                    activation='relu'))
    model.add(Dense(units=10, 
                    activation='softmax'))

    print(model.summary())

    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss='sparse_categorical_crossentropy',
                  metrics=['sparse_categorical_accuracy'])

    history = model.fit(x_data_train_norm.reshape(-1,28,28,1),
                        t_data_train,
                        epochs=200,
                        batch_size=100,
                        verbose=1,
                        validation_split=0.3
              )

    model.evaluate(x_data_test_norm.reshape(-1,28,28,1), t_data_test)

    
# GPU Memory 해제를 위한 코드이지만 잘 동작하는지는 확인이 필요    
tf.compat.v1.reset_default_graph()
tf.keras.backend.clear_session()
gc.collect()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dropout (Dropout)            (None, 576)               0

206/206 [==============================] - 2s 9ms/step - loss: 0.0102 - sparse_categorical_accuracy: 0.9966 - val_loss: 0.0480 - val_sparse_categorical_accuracy: 0.9887
Epoch 38/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0091 - sparse_categorical_accuracy: 0.9967 - val_loss: 0.0519 - val_sparse_categorical_accuracy: 0.9890
Epoch 39/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0085 - sparse_categorical_accuracy: 0.9972 - val_loss: 0.0463 - val_sparse_categorical_accuracy: 0.9901
Epoch 40/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0089 - sparse_categorical_accuracy: 0.9971 - val_loss: 0.0471 - val_sparse_categorical_accuracy: 0.9909
Epoch 41/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0073 - sparse_categorical_accuracy: 0.9975 - val_loss: 0.0575 - val_sparse_categorical_accuracy: 0.9892
Epoch 42/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0104 - sparse_categorical

206/206 [==============================] - 2s 9ms/step - loss: 0.0047 - sparse_categorical_accuracy: 0.9983 - val_loss: 0.0641 - val_sparse_categorical_accuracy: 0.9910
Epoch 83/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0071 - sparse_categorical_accuracy: 0.9978 - val_loss: 0.0608 - val_sparse_categorical_accuracy: 0.9895
Epoch 84/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0077 - sparse_categorical_accuracy: 0.9975 - val_loss: 0.0675 - val_sparse_categorical_accuracy: 0.9884
Epoch 85/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0074 - sparse_categorical_accuracy: 0.9976 - val_loss: 0.0546 - val_sparse_categorical_accuracy: 0.9915
Epoch 86/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0047 - sparse_categorical_accuracy: 0.9986 - val_loss: 0.0617 - val_sparse_categorical_accuracy: 0.9904
Epoch 87/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0043 - sparse_categorical

Epoch 127/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9991 - val_loss: 0.0812 - val_sparse_categorical_accuracy: 0.9889
Epoch 128/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0052 - sparse_categorical_accuracy: 0.9984 - val_loss: 0.0835 - val_sparse_categorical_accuracy: 0.9889
Epoch 129/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0043 - sparse_categorical_accuracy: 0.9983 - val_loss: 0.0746 - val_sparse_categorical_accuracy: 0.9905
Epoch 130/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0049 - sparse_categorical_accuracy: 0.9985 - val_loss: 0.0774 - val_sparse_categorical_accuracy: 0.9889
Epoch 131/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0026 - sparse_categorical_accuracy: 0.9993 - val_loss: 0.0792 - val_sparse_categorical_accuracy: 0.9889
Epoch 132/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0016 -

Epoch 172/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9990 - val_loss: 0.0757 - val_sparse_categorical_accuracy: 0.9910
Epoch 173/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0050 - sparse_categorical_accuracy: 0.9988 - val_loss: 0.0741 - val_sparse_categorical_accuracy: 0.9904
Epoch 174/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0018 - sparse_categorical_accuracy: 0.9995 - val_loss: 0.0820 - val_sparse_categorical_accuracy: 0.9905
Epoch 175/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0044 - sparse_categorical_accuracy: 0.9991 - val_loss: 0.0839 - val_sparse_categorical_accuracy: 0.9902
Epoch 176/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0036 - sparse_categorical_accuracy: 0.9987 - val_loss: 0.0717 - val_sparse_categorical_accuracy: 0.9900
Epoch 177/200
206/206 [==============================] - 2s 9ms/step - loss: 0.0032 -

NameError: name 'gc' is not defined